In [5]:
import import_ipynb
from yahoo_fin import options
from save_tickers import save_sp500_tickers
import pandas as pd
import datetime as dt
import os

def get_options(tickers=None, option_type=None, date=None):
    if not tickers:
        tickers = save_sp500_tickers()

    # set up a main dataframe for all tickers
    main_df = pd.DataFrame()

    # if option_type is None, fetch option data of all tickers
    if not option_type: 
        print('# start fetching all options')
        for i, ticker in enumerate(tickers):
            ticker = ticker.replace('.','')
            
            try:
                df = options.get_options_chain(ticker)
            except:
                print('Failed:', ticker)
                continue
            # iterate over both option types 
            for direction in df.keys():
                try: 
                    df = df[direction]
                except: 
                    continue 

                df.rename(columns={'Contract Name': 'contract_name', 
                                    'Last Trade Date': 'last_trade_date',
                                    'Stirke': 'stirke',
                                    'Last Price': 'last_price',
                                    'Bid': 'bid',
                                    'Ask': 'ask',
                                    'Change': 'change',
                                    '% Change': 'pct_change',
                                    'Volume': 'volume',
                                    'Open Interest': 'open_interest',
                                    'Implied Volatility': 'implied_volatility'}, inplace = True)
                df['ticker'] = ticker
                df['expiration_date'] = [row.strip(ticker)[:6] for row in df['contract_name'].values]
                df['expiration_date'] = ['20'+row[:2]+'/'+row[2:4]+'/'+row[4:] for row in df['expiration_date'].values]
                df['directions']= ['call' if row.strip(ticker)[6:7] == 'C' else 'put' for row in df['contract_name'].values]
                df.set_index('contract_name', inplace=True)
                # merge to main dataframe
                main_df = pd.concat([main_df,df], axis = 0, sort = False)
                # track progress
                print(i, ticker)
    # if specified option type 
    else:
        print('# start fetching {} options'.format(option_type))
        for i, ticker in enumerate(tickers):
            ticker = ticker.replace('.','')
            # get all put option data
            if option_type == 'put':
                try:
                    df = options.get_puts(ticker)
                except:
                    print('Failed:', ticker)
                    continue
            # get all call option data 
            elif option_type == 'call':
                try:
                    df = options.get_calls(ticker)
                except:
                    print('Failed:', ticker)
                    continue
            # rename columns to lower cases
            df.rename(columns={'Contract Name': 'contract_name', 
                                'Last Trade Date': 'last_trade_date',
                                'Stirke': 'stirke',
                                'Last Price': 'last_price',
                                'Bid': 'bid',
                                'Ask': 'ask',
                                'Change': 'change',
                                '% Change': 'pct_change',
                                'Volume': 'volume',
                                'Open Interest': 'open_interest',
                                'Implied Volatility': 'implied_volatility'}, inplace = True)
            df['ticker'] = ticker
            df['expiration_date'] = [row.strip(ticker)[:6] for row in df['contract_name'].values]
            df['expiration_date'] = ['20'+row[:2]+'/'+row[2:4]+'/'+row[4:] for row in df['expiration_date'].values]
            df['directions']= option_type
            df.set_index('contract_name', inplace=True)
            # merge to main dataframe
            main_df = pd.concat([main_df,df], axis = 0, sort = False)
            # track progress 
            print(i, ticker)

    if option_type:
        if not os.path.exists('sp500_{}'.format(option_type)):
            os.makedirs('sp500_{}'.format(option_type))
        main_df.to_csv('sp500_{}/{}_{}.csv'.format(option_type, dt.date.today(), option_type))
    else:
        if not os.path.exists('sp500_options'):
            os.makedirs('sp500_options')
        main_df.to_csv('sp500_all_options/{}_all_options.csv'.format(dt.date.today()))

    return main_df

if __name__ == '__main__':
    options = get_options(tickers=['AAPL'], option_type='put')
    print(options.head())



importing Jupyter notebook from save_tickers.ipynb
# start fetching put options
0 AAPL
                            last_trade_date  Strike  last_price  bid  ask  \
contract_name                                                               
AAPL200131P00190000  2020-01-29 11:58AM EST   190.0        0.01  0.0  0.0   
AAPL200131P00195000   2020-01-28 3:50PM EST   195.0        0.01  0.0  0.0   
AAPL200131P00200000   2020-01-28 1:00PM EST   200.0        0.01  0.0  0.0   
AAPL200131P00205000  2020-01-27 10:01AM EST   205.0        0.01  0.0  0.0   
AAPL200131P00210000   2020-01-28 3:57PM EST   210.0        0.01  0.0  0.0   

                     change pct_change volume open_interest  \
contract_name                                                 
AAPL200131P00190000     0.0          -      1             0   
AAPL200131P00195000     0.0          -      4             0   
AAPL200131P00200000     0.0          -      5             0   
AAPL200131P00205000     0.0          -      -             